In [20]:
import polars as pl
import scipy.io
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np 
from typing import Literal
from transphorm.preprocessors import MatFetcher
from transphorm.framework_helpers import dataloader_to_numpy
from transphorm.model_components import AATrialDataModule
from sklearn.model_selection import train_test_split
from sktime.clustering.k_means import TimeSeriesKMeans

In [16]:
path = Path("/Users/mds8301/Library/CloudStorage/OneDrive-NorthwesternUniversity/gaby_data/over_day_5")

In [17]:

def load_data(path: Path):
    X, y = dataloader_to_numpy(path)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    return X_train, X_test, y_train, y_test

In [18]:

X_train, X_test, y_train, y_test = load_data(path)

In [21]:
cluster = TimeSeriesKMeans(n_clusters=2)
cluster.fit(X_train)

TimeSeriesKMeans(n_clusters=2)

In [24]:
import joblib
path = "/Users/mds8301/Library/CloudStorage/OneDrive-NorthwesternUniversity/gaby_data/models/ts_cluster.joblib"
joblib.dump(cluster, path)

['/Users/mds8301/Library/CloudStorage/OneDrive-NorthwesternUniversity/gaby_data/models/ts_cluster.joblib']

In [25]:
joblib.load("/Users/mds8301/Library/CloudStorage/OneDrive-NorthwesternUniversity/gaby_data/models/ts_cluster.joblib")

TimeSeriesKMeans(n_clusters=2)